In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Term Deposit Valuation

Demonstrates pricing of a Term Deposit Investment.

Attributes
----------
instruments
valuation
lifecycle events
market data store
results store
quotes
"""

toggle_code("Hide docstring")

# Valuing and Rolling Term Deposits
In this notebook, we demonstrate how Term Deposits are handled in LUSID. We will show how to roll Term Deposits, represent cashflow payments and perform valuations.

Contents:
* [1. Initial Setup](#1.-Initial-Setup)
* [2. Create Portfolio](#2.-Create-Portfolio)
* [3. Create Term Deposit Instrument](#3.-Create-Term-Deposit-Instrument)
* [4. Add Transactions](#4.-Add-Transactions)
* [5. Get Holdings](#5.-Get-Holdings)
* [6. Rolling a Term Deposit](#6.-Rolling-a-Term-Deposit)
    * [6.1 Rolling Full Term Deposit Maturity Amount](#6.1-Rolling-full-term-deposit-maturity-amount)
    * [6.2 Rolling the Notional for the Term Deposit](#6.2-Rolling-the-Notional-for-the-Term-Deposit)
## 1. Initial Setup

In [2]:
# Import LUSID libraries
import lusid as lu
import lusid.models as lm
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam.refreshing_token import RefreshingToken
from IPython.display import display

# Import Libraries
from datetime import datetime, timedelta
import pytz
import pandas as pd
import json
import os

In [5]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

ValueError: The fields ['api_url', 'password', 'username', 'client_id', 'client_secret', 'token_url'] on the ApiConfiguration are set to None, please ensure that you have provided them directly, via a secrets file or environment variables

In [6]:
# Initiate the LUSID APIs required for the notebook
instruments_api = api_factory.build(lu.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.AggregationApi)
quotes_api = api_factory.build(lu.QuotesApi)

NameError: name 'api_factory' is not defined

In [ ]:
# Define scopes
scope = "ibor"
portfolio_code = "TermDepositExamplePortfolio"

# 2. Create Portfolio
In this section we will create a portfolio in LUSID to store the term deposits in.

In [ ]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(e.body)
#     print(json.loads(e.body)["title"])

## 3. Create Term Deposit Instrument
Next we must create an instrument to represent the Term Deposit.
In LUSID the rate of the Term Deposit must be defined at creation and cannot be adjusted.

In [ ]:
# Helper method to create Term Deposit instrument
def create_td_instrument(
    name,
    identifier,
    start_date,
    maturity_date,
    flow_convention,
    rate
):
    td_instrument = lm.TermDeposit(
        start_date=start_date,
        maturity_date=maturity_date,
        contract_size=1,
        flow_convention=flow_convention,
        rate=rate,
        dom_ccy="USD",
        instrument_type="TermDeposit",
        local_vars_configuration=None)

    td_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(identifier)},
        definition=td_instrument
    )

    upsert_request = {identifier: td_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    td_luid = upsert_response.values[identifier]
    print(f"Created instrument with LUID:{td_luid.lusid_instrument_id} and ClientInternal:{identifier}")
    return td_luid.lusid_instrument_id

The flow convention defines the payment schedule for the Term Deposit, this example pays every 6 months.

In [ ]:
flow_conventions = lm.FlowConventions(
    currency="USD",
    payment_frequency="6M",
    roll_convention="F",
    day_count_convention="Actual360",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0,
)

The Term Deposit Instrument A created below has a maturity time of 6 months and 3% interest rate.

In [ ]:
start_date = datetime(2020, 3, 18, 00, tzinfo=pytz.utc)
maturity_date_A = datetime(2020, 9, 18, 00, tzinfo=pytz.utc)

In [ ]:
td_client_a = "TD_Inst_A"
td_luid_a = create_td_instrument(
    name="Term Deposit Instrument A",
    identifier=td_client_a,
    start_date=start_date,
    maturity_date=maturity_date_A,
    flow_convention=flow_conventions,
    rate=0.03
)

## 4. Setup Transactions and Holdings
### 4.1 Transaction
In this section we will setup the initial transactions in the portfolio.

In [ ]:
# Helper method to upsert transactions into LUSID
def upsert_transaction(txn_id, type, luid, client_internal, txn_date, total_consideration):
    txn = lm.TransactionRequest(
    transaction_id=txn_id,
    type=type,
    instrument_identifiers={"Instrument/default/LusidInstrumentId": luid,
                            "Instrument/default/ClientInternal": client_internal},
    transaction_date=  txn_date.isoformat(),
    settlement_date=(txn_date+settlement_time).isoformat(),
    units=total_consideration,
    transaction_price=lm.TransactionPrice(price=1, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=total_consideration,currency="USD")
    )

    transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=[txn]
    )
    return view_transaction_in_lusid(txn_id)

def view_transaction_in_lusid(txn_id):
    df = lusid_response_to_data_frame(transaction_portfolios_api.get_transactions(scope,portfolio_code))
    return df[['transaction_id','type','instrument_identifiers.Instrument/default/ClientInternal','instrument_uid','transaction_price.price','total_consideration.amount','units','transaction_date','settlement_date']][df['transaction_id'] == txn_id]

In [ ]:
# In this notebook all transactions will have a 3 day settlement period.
settlement_time = timedelta(3)

We will first add an intial cash deposit of $1,000,000 to our portfolio.

In [ ]:
txn = lm.TransactionRequest(
    transaction_id="txn000",
    type="FundsIn",
    instrument_identifiers={"Instrument/default/Currency": "USD",
                            "Instrument/default/ClientInternal": "cash_USD"},
    transaction_date=(datetime(2020, 3, 10, 00, tzinfo=pytz.utc)).isoformat(),
    settlement_date=(datetime(2020, 3, 10, 00, tzinfo=pytz.utc) + settlement_time).isoformat(),
    units=1000000,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=1,currency="USD")
    )

resp = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[txn]
)

view_transaction_in_lusid("txn000")

The next transaction is Buying `Term Deposit Instrument A`, in LUSID this transaction represents investing $1,000,000 in a Term Deposit.

In [ ]:
upsert_transaction(
    txn_id="txn001", 
    type="Buy", 
    luid=td_luid_a, 
    client_internal=td_client_a, 
    txn_date=start_date, 
    total_consideration=1000000)

In [ ]:
# Helper method to view the transaction in LUSID up to a specific date
def get_transactions_todate(to_transaction_date):
    get_transactions_reponse = transaction_portfolios_api.get_transactions(
        scope = scope,
        code = portfolio_code,
        to_transaction_date=to_transaction_date
    )
    df = lusid_response_to_data_frame(get_transactions_reponse).sort_values('transaction_id', ascending=True)
    subdf = df[['transaction_id','type','instrument_identifiers.Instrument/default/ClientInternal','instrument_uid','total_consideration.amount','units','transaction_date','settlement_date','transaction_price.price']]
    return subdf.rename(columns={"instrument_identifiers.Instrument/default/ClientInternal":"ClientInternal","instrument_uid":"LusidInstrumentId"})

Running the cell below shows all the transactions that have occurred in the portfolio in LUSID at the start of `Term Deposit A`.

In [ ]:
get_transactions_todate(start_date)

## 4.2. Holdings
In this section we will view the portfolio holdings in LUSID.

In [ ]:
# Helper method to view holdings in LUSID
def get_holdings(date):
    resp = transaction_portfolios_api.get_holdings(scope=scope,
                                         code=portfolio_code,
                                         property_keys=["Instrument/default/ClientInternal","Instrument/default/Name"],
                                        effective_at=date)
    df = lusid_response_to_data_frame(resp)
    return df[["instrument_scope","instrument_uid","properties.Instrument/default/ClientInternal.value.label_value","properties.Instrument/default/Name.value.label_value","units","cost.amount","holding_type_name"]]

Running the cell below shows, after the initial buy transaction has settled `Term Deposit A` is in the portfolio. There is no cash balance as all the cash in the portfolio has been invested in `Term Deposit A`.

In [ ]:
get_holdings(start_date+settlement_time)

# 5. Running a Valuation
In this section we will setup and run valuations on our portfolio

## 5.1 Setup recipe
To run a valuation in LUSID we must first define a recipe.
The helper method below defines a recipe for a Term Deposit instrument by passing in `instrument_type="TermDeposit"` to the `VendorModelRule` parameter.

In [ ]:
def create_recipe(recipe_code, scope, model):
    # Populate recipe parameters
    recipe = lm.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    price_source="Lusid",
                    quote_type="Price",
                    field="mid",
                    quote_interval="1M"
                )
            ],
            options=lm.MarketOptions(
                attempt_to_infer_missing_fx=True,
                default_scope=scope,
                default_instrument_code_type="ClientInternal"
            )
        ),
        # Set the valuation model - curve with no discounting
        pricing=lm.PricingContext(
            model_rules=[
                lm.VendorModelRule(
                    supplier="Lusid",
                    model_name=model,
                    instrument_type="TermDeposit",
                    parameters="{}",
                )
            ],
            options=lm.PricingOptions(
                produce_separate_result_for_linear_otc_legs=False
            )
        ),
    )

    response = configuration_recipe_api.upsert_configuration_recipe(
                upsert_recipe_request=lm.UpsertRecipeRequest(
                    configuration_recipe=recipe
                )
            )

    return response

LUSID supports Simple Static, Discounting and Constant Time Value of Money pricing models for Term Deposits. In this notebook we will look at Simple Static and Constant Time Value of Money models.

In [ ]:
# Define recipe names
CTVoMRecipe = "TermDepositRecipeCTVoM"
SSRecipe = "TermDepositRecipeSS"

# Create recipes in LUSID
CTVoMResponse = create_recipe(CTVoMRecipe, scope, "ConstantTimeValueOfMoney")
SSResponse = create_recipe(SSRecipe, scope, "SimpleStatic")

In [ ]:
# Helper method runs a valuation on the portfolio for a given date and recipe
def perform_valuation(recipe, date):
    # Create valuation request
    valuation_request = lm.ValuationRequest(
        # Choose recipe to use
        recipe_id = lm.ResourceId(scope = scope, code = recipe),
        group_by = ['Instrument/default/ClientInternal'],
        metrics = [
        {"key": "Instrument/default/ClientInternal", "op": "Value"},
        {"key": "Instrument/default/Name", "op": "Value"},              # Reports the friendly name of the underlying instrument
        {"key": "Analytic/default/ValuationDate", "op": "Value"},       # Confirms the valuation date
        {"key": "Valuation/PV", "op": "Sum"}],                          # Calculates cost in GBP (the portfolio currency)
        # Identify portfolio to value
        portfolio_entity_ids = [lm.PortfolioEntityId(scope = scope, code = portfolio_code)],
        # Make date of valuation conditional on effective at date passed into function
        valuation_schedule = lm.ValuationSchedule(effective_at=date.isoformat()),
    )

    # Get portfolio valuation
    val_data = aggregation_api.get_valuation(valuation_request = valuation_request).data

    # Turn valuation response into pandas dataframe
    vals_df = pd.DataFrame(val_data)
    try:
        return vals_df.drop('Aggregation Errors', axis=1)
    except:
        return vals_df

Performing a valuation using Constant Time Value of Money (CTVoM) Model on the portfolio after the `Term Deposit A` settlement date, shows a valuation with a ~1.5% return of initial investment $1,000,000.
This is due to the 3% return defined on the Term Deposit and 6 month maturity with a 6 month payment schedule, so if the deposit was held until maturity it would pay out 1/2 of 3%.

In [ ]:
perform_valuation(CTVoMRecipe,start_date)

For Simple Static valuations in LUSID we must provide a quote for the instrument. The date of the quote should be within 5 days of the valuation date.
In this example we will set the quote and valuation date at the start of the `Term Deposit A`.

In [ ]:
# Helper method to upsert a quote
def upsert_quote(client_internal, date, rate):
    quote_request = {
       "quote_request" : lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=client_internal,
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=date.isoformat(),
            ),
            metric_value=lm.MetricValue(value=rate, unit="USD"),
        )}

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(scope=scope, request_body=quote_request)
    display( f"Successfully upserted quote for instrument: {response.values['quote_request'].quote_id.quote_series_id.instrument_id} with date:{date}")

 The quote defined below has a rate of 6%, this is defined by setting `value=1.06`.

In [ ]:
upsert_quote(td_client_a, start_date, 1.06)

Performing valuation using Constant Time Value of Money Model on the portfolio after the `Term Deposit A` settlement date shows a valuation with a ~6% return of initial investment $1,000,000.
This is due to the 6% return defined on the quote definition above.

In [ ]:
perform_valuation(SSRecipe,start_date)

## 6. Rolling a Term Deposit
In LUSID we handle rolling a Term Deposit by liquidating the Term Deposit before maturity and buying it back again in the form of new Term Deposit Instrument.
This section will show 2 example scenarios:
1. Rolling full term deposit maturity amount
2. Rolling the notional for the term deposit


### 6.1 Rolling Full Term Deposit Maturity Amount
In this example when a Term Deposit reaches maturity, we will roll the Term Deposit and reinvest all of the Term Deposit's return into another Term Deposit. The total invested in a Term Deposit will increase due to the interest returned at maturity.

#### 6.1.1 Liquidate Term Deposit at Maturity
When Term Deposit reaches maturity we will receive a payment of the `initial amount invested + interest`.
In LUSID we can call the `get_upsertable_portfolio_cash_flows` to get a transaction which represents the cash returned to the portfolio when a Term Deposit reaches maturity.
In this notebook we will always use CTVoM model for cashflow calculations.

In [ ]:
# Helper method to get cashflow in the portfolio over a given time period from start date to maturity.
def get_cashflow(start_date, maturity_date, recipe=CTVoMRecipe):
    resp = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
        scope=scope,
        code=portfolio_code,
        effective_at=maturity_date- timedelta(days=1),
        window_start=start_date,
        window_end=maturity_date + timedelta(days=2),
        recipe_id_scope=scope,
        recipe_id_code=recipe
    )
    return resp.values

Next we get the cashflow for `Term Deposit A` at maturity and upsert the transaction to LUSID.
The cashflow for `Term Deposit A`, with LUID `LUID_00003DET`, shows a total consideration with a 1.5% return of the initial amount invested. This is made up of \$$ 1,000,000 from the initial investement and \$ 15,333.33 accrued interest.

In [ ]:
cashflow_txn = get_cashflow(start_date, maturity_date_A)
response = transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=cashflow_txn
)
view_transaction_in_lusid(cashflow_txn[0].transaction_id)

Now that `Term Deposit Instrument A` has matured we will remove this instrument from the portfolio. The `StockOut` transaction type has no effect on the cash balance, it just moves the instrument out of the portfolio.

In [ ]:
upsert_transaction(
    txn_id="txn002", 
    type="StockOut", 
    luid=td_luid_a, 
    client_internal=td_client_a, 
    txn_date=maturity_date_A, 
    total_consideration=1000000)

### 6.1.2 Reinvest the Matured Term Deposit's Returns
To represent rolling a Term Deposit in this section we will buy a new Term Deposit Instrument.
The new instrument defined below, `Term Deposit B`, will have a new start and maturity date but will have the same maturity time as `Term Deposit A` and an interest rate of 4%.

In [ ]:
maturity_date_B = datetime(2021, 3, 18, 00, tzinfo=pytz.utc)

In [ ]:
td_client_b = "TD_Inst_B"
td_luid_b  = create_td_instrument(
    name="Term Deposit Instrument B",
    identifier=td_client_b,
    start_date=maturity_date_A,
    maturity_date=maturity_date_B,
    flow_convention=flow_conventions,
    rate=0.04
)

We will now invest all of the cash returned from `Term Deposit A` into `Term Deposit B`.
In LUSID we show this by buying `Term Deposit B` instrument, with the total consideration equalling the returns from `Term Deposit A`.

In [ ]:
inst_A_cashflow_amount = cashflow_txn[0].total_consideration.amount

upsert_transaction(
    txn_id="txn003", 
    type="Buy", 
    luid=td_luid_b, 
    client_internal=td_client_b, 
    txn_date=maturity_date_A, 
    total_consideration=inst_A_cashflow_amount)

The transactions table below shows the transactions in the portoflio that have happened by the maturity date of `Term Deposit A`.
We can see the `StockOut` transaction removing `Term Deposit A` from the portfolio at maturity and the `CashFlow` transaction which realises the cash value of the deposit. The `Buy` transaction represents reinvesting $1,015,333.3333 in `Term Deposit B`.

In [ ]:
get_transactions_todate(maturity_date_A)

The table below shows the holdings in the portfolio after the transactions have settled 3 days later.
Here we can see the `Term Deposit Instrument A` is no longer in the portfolio and has been replaced by `Term Deposit Instrument B`.

In [ ]:
get_holdings(maturity_date_A + settlement_time)

The valuation below shows the valuation for `Term Deposit Instrument B` using the CTVoM model. This value is higher than the CTVoM valuation for `Term Deposit Instrument A` due to the higher initial investment.

In [ ]:
perform_valuation(CTVoMRecipe, maturity_date_A + settlement_time)

To perform another Simple Static valuation we must upsert a new quote for the new `Term Deposit instrument B` at the roll start date.
The Simple Static valuation below wih a 6% return of the initial investment.

In [ ]:
upsert_quote(td_client_b, maturity_date_A, 1.06)
upsert_quote(td_client_a, maturity_date_A, 1.06)

In [ ]:
perform_valuation(SSRecipe,maturity_date_A)

## 6.2 Rolling the Notional for the Term Deposit
In this example when a Term Deposit reaches maturity, we will roll a Term Deposit and reinvest $1,000,000 in another Term Deposit with a higher interest rate.

### 6.2.1 Liquidate the Term Deposit at Maturity
As in the previous example, we first upsert the cashflow transaction. The dates for the cashflow are over the investment period of `Term Deposit B`.

In [ ]:
txn = get_cashflow(maturity_date_A + settlement_time, maturity_date_B, CTVoMRecipe)
response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=txn
)
view_transaction_in_lusid(txn[0].transaction_id)

Next we remove the `Term Deposit B` instrument from our portfolio.

In [ ]:
upsert_transaction(
    txn_id="txn004", 
    type="StockOut", 
    luid=td_luid_b, 
    client_internal=td_client_b, 
    txn_date=maturity_date_B, 
    total_consideration=inst_A_cashflow_amount)

### 6.2.2 Buy Term Deposit back with an Increased Rate
To roll the term deposit we will create a new instrument, `Term Deposit C`. It will have the same maturity time as the previous Term Deposits, with a higher rate of 5% and a new start and end date.

In [ ]:
maturity_date_C = datetime(2021, 9, 18, 00, tzinfo=pytz.utc)

In [ ]:
# Define a new instrument with a higher rate
td_client_c = "TD_Inst_C"
td_luid_c = create_td_instrument(
    name="Term Deposit Instrument C",
    identifier=td_client_c,
    start_date=maturity_date_B,
    maturity_date=maturity_date_C,
    flow_convention=flow_conventions,
    rate=0.05
)

In this example we will only invest $1,000,000 into `Term Deposit C` and leave the remaining cash returned from `Term Deposit B` in the portfolio.

In [ ]:
upsert_transaction(
    txn_id="txn005", 
    type="Buy", 
    luid=td_luid_c, 
    client_internal=td_client_c, 
    txn_date=maturity_date_B, 
    total_consideration=1000000)

The cell below shows the transactions in the portfolio after Term Deposit B has been purchased

In [ ]:
get_transactions_todate(maturity_date_B)

The table below shows the holdings in the portfolio after the transactions have settled 3 days later.
Here we can see the `Term Deposit Instrument B` is no longer in the portfolio and has been replaced by `Term Deposit Instrument C` and there is an excess cash balance of $35,752.8148. The cash balance represents the interest returned from `Term Deposit B` which was not reinvested in `Term Deposit C`.

In [ ]:
get_holdings(maturity_date_B + timedelta(days=3))

The valuation below shows a PV with a ~2.5% return of the initial $1,000,000 invested in `Term Deposit C`. This reflects the 5% interest rate over the 6 month Term Deposit where 5/2= 2.5.

In [ ]:
perform_valuation(CTVoMRecipe, maturity_date_B)

To get a Simple Static valuation we will first upsert a quote for the new Term Deposit. This quote has an interest rate of 4%, we can see this reflected in a valuation with a 4% return on the initial investment. The interest rate of the `Term Deposit C` instrument does not effect the Simple Static valuation.

In [ ]:
upsert_quote(td_client_b, maturity_date_B, 1.04)
upsert_quote(td_client_c, maturity_date_B, 1.04)

In [ ]:
perform_valuation(SSRecipe, maturity_date_B)